In [ ]:
from google.colab import drive
import sys
import os

In [ ]:
!/opt/bin/nvidia-smi

In [ ]:
!pip install transformers

In [ ]:
drive.mount()
path = ''
os.chdir(path)
os.listdir(path)
import sys
sys.path.append()

In [ ]:
from process_pseudo_data import add_pseudo_data_to_train,get_pseudo_data_for_predict,ncz_process
from predict import predict
from utils import Params
from postprocess import postprocess
from pathlib import Path
from preprocess import get_train_val,get_testset

import os
import random
# 参数解析器
import argparse
import logging
from tqdm import trange

import utils
import torch

from optimization import BertAdam
# from utils import FGM
from evaluate import evaluate
from dataloader import NERDataLoader
from model import BertForTokenClassification

In [ ]:
train_path = Path('./data/ccks2021_train/ner_train.txt')

save_dir = Path('./data')
get_train_val(train_path, save_dir)

In [ ]:
#ncz数据处理
ncz_process()
#将伪标签数据构造为待预测
get_pseudo_data_for_predict(mode='ccks')

In [ ]:
# 参数解析器
parser = argparse.ArgumentParser()
# 设定参数
parser.add_argument('--seed', type=int, default=2020, help="random seed for initialization")
parser.add_argument('--ex_index', type=int, default=1, help="实验名称索引")
parser.add_argument('--device_id', type=int, default=0, help="GPU index")
parser.add_argument('--restore_file', type=str, default='best', required=False,
                    help="Optional, name of the file containing weights to reload before training")
parser.add_argument('--mode', default='test', help="'val', 'test' or 'pseudo'")
parser.add_argument('--pre_model_type', type=str, help="预训练模型类型")
parser.add_argument('--ds_encoder_type', type=str, help="下游编码器类型")

args = parser.parse_args(args = ['--seed','2020','--ex_index','1',
                                 '--device_id','0',
                                 '--restore_file','best',
                                 '--mode','pseudo',
                                 '--pre_model_type','RoBERTa',
                                 '--ds_encoder_type','LSTM'])

In [ ]:
params = utils.Params(args.pre_model_type, args.ex_index)
params.fusion_layers

In [ ]:
# 构建伪标签
params = utils.Params(args.pre_model_type, args.ex_index)
# set type
params.ds_encoder_type = args.ds_encoder_type

# 预测验证集还是测试集
mode = args.mode
# Set the random seed for reproducible experiments
random.seed(args.seed)
torch.manual_seed(args.seed)
params.seed = args.seed

# Set the logger
utils.set_logger()

# get dataloader
dataloader = NERDataLoader(params)

# Define the model
logging.info('Loading the model...')
# Reload weights from the saved file
model, optimizer = utils.load_checkpoint(os.path.join(params.model_dir, args.restore_file + '.pth.tar'))
model.to(params.device)
logging.info('- done.')

logging.info("Loading the dataset...")
loader = dataloader.get_dataloader(data_sign=mode)
logging.info('-done')

logging.info("Starting prediction...")
# Create the input data pipeline
predict(model, loader, params, mode)
logging.info('-done')

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--ex_index', type=int, default=1, help="实验名称索引")
parser.add_argument('--mode', type=str, default='test', help="后处理结果类型")

args = parser.parse_args(args = ['--ex_index','1',
                                 '--mode','pseudo'])

params = Params(ex_index=args.ex_index)
postprocess(params, mode=args.mode)

#将伪标签数据追加到训练集
add_pseudo_data_to_train()